In [107]:
import pyspark 
import argparse 
import json 

# 4.1.1 E: Top n frequent words in the review text

sc_conf = pyspark.SparkConf()
sc_conf.setAppName('task1')  
sc_conf.setMaster('local[*]') 
sc_conf.set('spark.driver.memory', '8g')
sc_conf.set('spark.executor.memory', '4g')
# sc = pyspark.SparkContext(conf=sc_conf)
sc = pyspark.SparkContext.getOrCreate()
sc.setLogLevel("OFF")

# read in the lines and create the rdd 
lines = sc.textFile("./data/review.json")

# loading the json into a python object 
rdd = lines.map(lambda line: json.loads(line))

In [108]:
punctuations = ["(", "[", ",", ".", "!", "?", ":", ";", "]", ")"]

In [109]:
stopwords = []
with open('./data/stopwords') as f: 
    stopwords.append(f.read().split('\n'))
print(stopwords[0:10]) 

[['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'n

In [110]:
text = rdd.map(lambda x: x['text'].replace('!', '').replace('(', '').replace("[","").replace(',','').replace('.', '').replace('?', '').replace(':','').replace(';',''))
# first we need to filter by character 
text = text.map(lambda x: x.lower().split(" "))


In [111]:
text.take(4)

[['walked',
  'in',
  'around',
  '4',
  'on',
  'a',
  'friday',
  'afternoon',
  'we',
  'sat',
  'at',
  'a',
  'table',
  'just',
  'off',
  'the',
  'bar',
  'and',
  'walked',
  'out',
  'after',
  '5',
  'min',
  'or',
  'so',
  "don't",
  'even',
  'think',
  'they',
  'realized',
  'we',
  'walked',
  'in',
  'however',
  'everyone',
  'at',
  'the',
  'bar',
  'noticed',
  'we',
  'walked',
  'in',
  'service',
  'was',
  'non',
  'existent',
  'at',
  'best',
  'not',
  'a',
  'good',
  'way',
  'for',
  'a',
  'new',
  'business',
  'to',
  'start',
  'out',
  'oh',
  'well',
  'the',
  'location',
  'they',
  'are',
  'at',
  'has',
  'been',
  'about',
  '5',
  'different',
  'things',
  'over',
  'the',
  'past',
  'several',
  'years',
  'so',
  'they',
  'will',
  'just',
  'be',
  'added',
  'to',
  'the',
  'list',
  'smdh'],
 ['michael',
  'from',
  'red',
  'carpet',
  'vip',
  'is',
  'amazing',
  '',
  'i',
  'reached',
  'out',
  'because',
  'i',
  'needed',
  

In [112]:
top_n = text.filter(lambda x: x not in stopwords).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b).sortBy(lambda x: x[1], False)
    

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 130.0 failed 1 times, most recent failure: Lost task 10.0 in stage 130.0 (TID 1286) (192.168.250.233 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/lucky/miniconda3/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/lucky/miniconda3/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/lucky/miniconda3/

In [ ]:
print(top_n.take(5))

[('', 6), ("if i could give this company less than one star i would i am a professional live streamer and online entertainer through twitchtv i have had my personal information leaked on the internet and this company does nothing to protect me or my information i have had two accounts through them one under my own information and one under a friend's information both times a stranger accessed my accounts with nothing but my address or name they give out information over the phone like account holder's name phone number and the last four of your social with only a single identifying measure if someone knows your name address and phone number or 2/3 they can access your account and get your personal information this company will give out and change passwords and pin numbers over the phone they gave out the last four numbers of my ssn do not use this company if you value your identity after your information has been leaked they do nothing to protect you", 4), ("i love oregano's and they s

[("received a  mailer and thought okay let's go. walked in on a monday afternoon greeted by a bartender wearing a cute  t shirt that said the bar and below that the baby. how cute however never got her name. looked over menu asked what was good she said the club. so ordered it and  yes it is good. had a beer a club on white toast served with fries and watched sport center. really felt at ease there and good music playing. will go back again.",
  1)]

[['walked',
  'in',
  'around',
  '4',
  'on',
  'a',
  'friday',
  'afternoon,',
  'we',
  'sat',
  'at',
  'a',
  'table',
  'just',
  'off',
  'the',
  'bar',
  'and',
  'walked',
  'out',
  'after',
  '5',
  'min',
  'or',
  'so.',
  "don't",
  'even',
  'think',
  'they',
  'realized',
  'we',
  'walked',
  'in.',
  'however',
  'everyone',
  'at',
  'the',
  'bar',
  'noticed',
  'we',
  'walked',
  'in!!!',
  'service',
  'was',
  'non',
  'existent',
  'at',
  'best.',
  'not',
  'a',
  'good',
  'way',
  'for',
  'a',
  'new',
  'business',
  'to',
  'start',
  'out.',
  'oh',
  'well,',
  'the',
  'location',
  'they',
  'are',
  'at',
  'has',
  'been',
  'about',
  '5',
  'different',
  'things',
  'over',
  'the',
  'past',
  'several',
  'years,',
  'so',
  'they',
  'will',
  'just',
  'be',
  'added',
  'to',
  'the',
  'list.',
  'smdh!!!']]

In [ ]:
counts = splitting_words.flatMap(lambda line: line.split(' ')) \
    .filter(lambda x: x not in stopwords) \
    .map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)

In [ ]:
counts = splitting_words.flatMap(lambda line: line.split(' ')) \
    .filter(lambda x: x not in stopwords) 


In [ ]:
print(counts.take(3))
# counts = counts.filter(lambda x: x not in stopwords) 
# print(counts.collect()[0:3])    

['walked', 'in', 'around']


In [ ]:
# counts = counts.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)
# print(counts.collect()[0:3])    

In [ ]:
final = counts.sortBy(lambda x: x[1], False)
print(counts.collect()[0:3])    

[('even', 215856), ('they', 994327), ('noticed', 18323)]


In [ ]:
# print(final.collect()[0:3])